# <font size=6 color=blackblue>RESTART</font>
---

In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Colab Start Alive</font>

# @markdown Bağlantınız kesildiğinde otomatik olarak yeniden bağlan düğmesine basacak bir JS kodu çalıştırır.

# @markdown This kid runs a JS code that will print your idle disconnect automatically load reload.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

#<img src='https://i.ibb.co/nD9NnTc/drive.png' height="50" alt="GDrive"/>

In [ ]:
#@markdown <h2><b>←</b>Mount your Google Drive</h2>
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@markdown <h2><b>←</b> ឵឵Mount Another Google Drive</h2>
#To mount another drive:
 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!sudo mkdir /content/drive1
!google-drive-ocamlfuse /content/drive1

In [ ]:
#@markdown <h4><b>←</b><b>Force Remount</b> your Google Drive</h4>
drive.mount("/content/drive", force_remount=True)

# <img src='https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png' height="46" alt="rclone"/>

### ← Mount Cloud

In [1]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 1. </b>Upload your <code>rclone.conf</code> config</font> { vertical-output: true }
## @markdown <h7>Your <code>rclone.conf</code> file is located at:</i></h4>
# ================================================================ #
MODE = "RCONFIG"
from os import path as _p
from IPython.display import clear_output

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/RLabClone/master/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("Upload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()

Select file from your computer.



Saving rclone.conf to rclone.conf
Upload completed.


In [ ]:
# ============================= FORM ============================= #
#@title <font size="4"><b>Step 2</b>. Mount Your Cloud </font> { vertical-output: true }
Cache_Directory = "DISK"
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#127ACC;margin-top:0px;">
                  Choose Drive to Mount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
# @markdown #### ⬅️ Uzaktan Kumandayı Takmak / Kaldırmak İçin Tıklayın
Translated from: Portuguese (Google)
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Montar", MountCMD, "primary"),
          MakeButton("Desmontar", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"🤩Beleza! Drive Montado👍! - \t{mPoint}")
    else:
      print(f"🤔 Deu alguma parada errada! Tente de novo! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Pronto! 💣 Desmontado - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Rclone.conf dosyanızı bilgisayarınıza indirin.
Translated from: Portuguese (Google) 🔽
MODE = "RCONFIG"
# ================================================================ #
from google.colab import files

def downloadFile():
  if MODE == "UTILS":
      filePath = "/root/.ipython/rlab_utils.py"
  elif MODE == "RCONFIG":
      filePath = f"{PATH_RClone_Config}/rclone.conf"
  else:
      pass
  try:
    files.download(filePath)
  except FileNotFoundError:
    print("Vixe!! Kadê o arquivo? 🤦‍♂️")

if __name__ == "__main__":
  downloadFile()

#  <font color=red size=6><b>**MEGA TRANSFER**</b></font>

##__Transfer from Mega to GDrive__





In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Mega to GDrive</h2></center><br>
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")
 
from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "https://mega.nz/folder/pAoAiBAD#RqyIdL5dadadd" #@param {type:"string"}
OUTPUT_PATH = "/content/drives/TOR" #@param {type:"string"}
# @markdown ##### Sometimes this cell doesn't stop itself after the completion of the transfer. In case of that stop the cell manually.
# @markdown ##### Bazen bu hücre aktarım tamamlandıktan sonra kendini durdurmaz. Bu durumda hücreyi manuel olarak durdurun.
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()
 
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']
 
def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out
 
 
def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        
 
 
transfare()

####_Update: If you have a Mega Pro account, sign in to use its bandwidth(transfer quota)_
####_Güncelleme: Bir Mega Pro hesabınız varsa, bant genişliğini (aktarım kotası)_ kullanmak için oturum açın

In [ ]:
# @markdown <center><h3>Mega ID</h3></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex
 
 
class TimeoutError(Exception):
    pass
 
 
def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)
 
        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result
 
        return wraps(func)(wrapper)
 
    return decorator
 
 
if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split
 
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh
 
@timeout(10)
def runShT(args):
    return runSh(args, output=True)
 
 
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd
 
# INPUT YOUR MEGA ID
 
USERNAME = "mokav97043@94jo.com"  # @param {type:"string"}
PASSWORD = "kamileecher12345"  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Mega Login")
# @markdown After signing in, use the above cell(Main Cell) to transfer files from Mega to GDrive using the transfer quota of your Mega Pro Account

# @markdown Oturum açtıktan sonra, Mega Pro Hesabınızın aktarım kotasını kullanarak Mega'dan GDrive'a dosya aktarmak için yukarıdaki hücreyi (Ana Hücre) kullanın.

# Rclone Multicloud

In [ ]:
Supported storage providers:

  • 1Fichier (https://rclone.org/fichier/)
  • Alibaba Cloud Object Storage (https://rclone.org/s3/#alibaba-oss)
  • Amazon Drive (https://rclone.org/amazonclouddrive/)
  • Amazon S3 (https://rclone.org/s3/)
  • Backblaze B2 (https://rclone.org/b2/)
  • Box (https://rclone.org/box/)
  • Ceph (https://rclone.org/s3/#ceph)
  • Citrix ShareFile (https://rclone.org/sharefile/)
  • DigitalOcean Spaces (https://rclone.org/s3/#digitalocean-spaces)
  • Dreamhost (https://rclone.org/s3/#dreamhost)
  • Dropbox (https://rclone.org/dropbox/)
  • Enterprise File Fabric (https://rclone.org/filefabric/)
  • FTP (https://rclone.org/ftp/)
  • GetSky (https://rclone.org/jottacloud/)
  • Google Cloud Storage (https://rclone.org/googlecloudstorage/)
  • Google Drive (https://rclone.org/drive/)
  • Google Photos (https://rclone.org/googlephotos/)
  • HDFS (https://rclone.org/hdfs/)
  • HTTP (https://rclone.org/http/)
  • Hubic (https://rclone.org/hubic/)
  • Jottacloud (https://rclone.org/jottacloud/)
  • IBM COS S3 (https://rclone.org/s3/#ibm-cos-s3)
  • Koofr (https://rclone.org/koofr/)
  • Mail.ru Cloud (https://rclone.org/mailru/)
  • Memset Memstore (https://rclone.org/swift/)
  • Mega (https://rclone.org/mega/)
  • Microsoft Azure Blob Storage (https://rclone.org/azureblob/)
  • Microsoft OneDrive (https://rclone.org/onedrive/)
  • Minio (https://rclone.org/s3/#minio)
  • Nextcloud (https://rclone.org/webdav/#nextcloud)
  • OVH (https://rclone.org/swift/)
  • OpenDrive (https://rclone.org/opendrive/)
  • OpenStack Swift (https://rclone.org/swift/)
  • Oracle Cloud Storage (https://rclone.org/swift/)
  • ownCloud (https://rclone.org/webdav/#owncloud)
  • pCloud (https://rclone.org/pcloud/)
  • premiumize.me (https://rclone.org/premiumizeme/)
  • put.io (https://rclone.org/putio/)
  • QingStor (https://rclone.org/qingstor/)
  • Rackspace Cloud Files (https://rclone.org/swift/)
  • Scaleway (https://rclone.org/s3/#scaleway)
  • Seafile (https://rclone.org/seafile/)
  • SeaweedFS (https://rclone.org/s3/#seaweedfs)
  • SFTP (https://rclone.org/sftp/)
  • StackPath (https://rclone.org/s3/#stackpath)
  • SugarSync (https://rclone.org/sugarsync/)
  • Tardigrade (https://rclone.org/tardigrade/)
  • Tencent Cloud Object Storage (https://rclone.org/s3/#tencent-cos)
  • Wasabi (https://rclone.org/s3/#wasabi)
  • WebDAV (https://rclone.org/webdav/)
  • Yandex Disk (https://rclone.org/yandex/)
  • Zoho WorkDrive (https://rclone.org/zoho/)

# Telegram @kamileecher